In [1]:
import json
import copy

In [2]:
# pretty prints
from pprint import pprint

In [3]:
# creating directory for json configs
import os

if not os.path.isdir("gobot"):
    os.mkdir("gobot")

In [4]:
import deeppavlov

注意:“go_bot”模型在CPU上运行得更快，所以我们忽略现有的gpu:

In [5]:
!export CUDA_VISIBLE_DEVICES=""
!echo "cuda visible devices = '"$CUDA_VISIBLE_DEVICES"'"

cuda visible devices = ''


# 混合面向目标的机器人

对话机器人分为两类:  
1. 面向目标的模型  
那些必须在谈话结束时达到某种目标的模型:
 * 餐厅和机票预订
 * 客户支持服务系统
 * 其他
2. 聊天模型  
那些只是为了好玩而聊天的人，和你聊天的时间越长，聊天的效果越好，例如:
 * "replica"移动应用程序  
  
  在这里，我们将只深入研究面向目标的任务。

![](https://github.com/deepmipt/DeepPavlov/raw/c7896c6db96f43f57cacd9a6a471e37cb70bf07a/examples/tutorials/img/bot_architecture00.png)

一个经典的对话系统包括:  
1. 自然语言理解组件(NLU)  
作用：理解人们所说的话，并把它的“理解”转化成机器可读的形式。  
输入输出：它以表达的文本作为输入，并转换为对话“框架”。  
“框架”主要包括：  
 * 域值 domain value(域是“对话类型”);
 * 目的值 intent value(人类话语的意图：“欢迎”，“询问”等)
 * 实体槽 entity slots(人类的“地点”、“时间”等)
  
2. 对话管理器组件(DM)  
作用：决定如何回应。  
输入输出：它需要一个由NLU框架输入和输出操作(它不是最终文本，它是一个标签label)。  
例如，有些操作："say_welcome", "say_goodbye", "ask_location","give_weather"等等。  
  
3. 自然语言生成组件(NLG)  
作用：将DM输出的操作转换为实际的文本响应表示形式。  
例如，"say_goodbye" -> "You are welcome!".


## NLU
![](https://github.com/deepmipt/DeepPavlov/raw/c7896c6db96f43f57cacd9a6a471e37cb70bf07a/examples/tutorials/img/bot_architecture01.png)

让我们考虑一个带有NLU组件的对话系统，它由单个命名实体识别组件(NER)组成。  
前面的一个教程介绍了deeppavlov的NER模型，并展示了如何使用它。

## DM & NLG
本教程主要讨论如何实现：
* 对话管理组件
* 自然语言生成组件

### 数据集
我们将会在“Dialog State Tracking Chellenge 2”这个数据集上训练我们的聊天机器人。  
我们先把它下载下来：

In [6]:
from deeppavlov.dataset_readers.dstc2_reader import DSTC2Version2DatasetReader

data = DSTC2Version2DatasetReader().read(data_path="tmp/my_download_of_dstc2")

2018-08-22 16:45:15.277 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 214: [loading dialogs from tmp/my_download_of_dstc2/dstc2-trn.jsonlist]
2018-08-22 16:45:15.525 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 214: [loading dialogs from tmp/my_download_of_dstc2/dstc2-val.jsonlist]
2018-08-22 16:45:15.661 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 214: [loading dialogs from tmp/my_download_of_dstc2/dstc2-tst.jsonlist]


DSTC2DatasetReader下载了所需的数据并保存到磁盘。  
DialogDatasetIterator将数据作为输入并将其转换为批处理。

In [7]:
from deeppavlov.dataset_iterators.dialog_iterator import DialogDatasetIterator

batches_generator = DialogDatasetIterator(data, seed=1443, shuffle=True)\
                                         .gen_batches(batch_size=4, data_type='train')

让我们仔细看看批处理内容:

In [8]:
batch = batches_generator.__next__()

每个批处理是两个元素的元组:
*  x的列表
*  y的列表

In [9]:
x_batch, y_batch = batch

x_batch(和y_batch)由4个样例组成。这是因为batch_size是4。

In [10]:
len(x_batch)

4

一个样例是一个对话，对话会转换成类似这样：

In [11]:
dialog_id = 0
dialog = [(x, y) for x, y in zip(x_batch[dialog_id], y_batch[dialog_id])]

turn_id = 0
print("----{}th turn----".format(turn_id)) 
pprint(dialog[turn_id], indent=8)


----0th turn----
(       {'intents': [], 'prev_resp_act': None, 'text': ''},
        {       'act': 'welcomemsg',
                'text': 'Hello, welcome to the Cambridge restaurant system. '
                        'You can ask for restaurants by area, price range or '
                        'food type. How may I help you?'})


这是整个对话的样子(打印x_batch和y_batch的‘text’部分):

In [12]:
for turn in dialog:
    x, y = turn
    print('::', x['text']) 
    print('>>', y['text'], '\n')

:: 
>> Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you? 

:: traditional
>> api_call area="dontcare" food="traditional" pricerange="dontcare" 

:: traditional
>> I am sorry but there is no traditional restaurant that matches your request. 

:: italian food
>> What part of town do you have in mind? 

:: south
>> api_call area="south" food="italian" pricerange="dontcare" 

:: south
>> Pizza hut cherry hinton is a nice place in the south of town serving tasty italian food. 

:: phone number
>> The phone number of pizza hut cherry hinton is 01223 323737. 

:: post code
>> The post code of pizza hut cherry hinton is C.B 1, 7 D.Y. 

:: thank you good bye
>> You are welcome! 



### DeepPavlov配置
DeepPavlov使用json配置数据管道。  

配置包括以下部分:  
* dataset_reader  
  * 配置数据集阅读器组件(负责下载数据并保存到磁盘);  
* dataset_iterator
  * 配置数据集迭代器组件(制作数据批次(序列)发送到管道组件中);
* metadata
  * 额外信息(数据下载 和 电报配置 的url);
* train
  * 训练过程配置(批次大小、训练周期数等);
* chainer
  * 指定数据流(运行哪些组件以及以何种顺序运行);  
    
    
让我们构造一个简单的配置来创建输入示例token的字典

In [13]:
vocab_config = {}

* dataset_reader -- 配置数据集阅读器组件(负责下载数据并保存到磁盘);  

In [14]:
dstc2_reader_comp_config = {
    'name': 'dstc2_v2_reader',
    'data_path': 'dstc2_v2'
}

In [15]:
vocab_config['dataset_reader'] = dstc2_reader_comp_config

* dataset_iterator -- 配置数据集迭代器组件(制作数据批次(序列)发送到管道组件中);

In [16]:
dialog_iterator_comp_config = {
    'name': 'dialog_iterator'
}

In [17]:
vocab_config['dataset_iterator'] = dialog_iterator_comp_config

In [18]:
vocab_config

{'dataset_reader': {'name': 'dstc2_v2_reader', 'data_path': 'dstc2_v2'},
 'dataset_iterator': {'name': 'dialog_iterator'}}

* metadata -- 额外信息  
  * metadata.download -- 为了让配置生效而应该下载的数据列表;

In [19]:
dstc2_download_config = {
    'url': 'http://files.deeppavlov.ai/datasets/dstc2_v2.tar.gz',
    'subdir': 'dstc2_v2'
}

In [20]:
vocab_config['metadata'] = {}
vocab_config['metadata']['download'] = [
    dstc2_download_config
]

In [21]:
vocab_config

{'dataset_reader': {'name': 'dstc2_v2_reader', 'data_path': 'dstc2_v2'},
 'dataset_iterator': {'name': 'dialog_iterator'},
 'metadata': {'download': [{'url': 'http://files.deeppavlov.ai/datasets/dstc2_v2.tar.gz',
    'subdir': 'dstc2_v2'}]}}

* train -- 训练过程配置(批次大小、训练周期数等);  
我们现在不需要训练任何东西，只需要构建一个字典(适合整个数据集一次)，所以“train”部分是空的。

In [22]:
vocab_config['train'] = {}

* chainer代表数据流:  
  * chainer.in -- 是输入样本名称的列表(一个数据样本可能包含多个变量);
  * chainer.in_y -- 是输入标签名称的列表(每个样本可能有不同种类的标签);
  * chainer.out -- 是输出预测名称的列表(通常与“chainer.in_y”具有相同的长度);  
  
X在这里只是一个表达(utterance)  
  

Y是空的（我们不需要像训练神经网络那样训练字典）  
  
对配置没有预测，没有什么可预测的。

In [23]:
vocab_config['chainer'] = {}
vocab_config['chainer']['in'] = ['utterance']
vocab_config['chainer']['in_y'] = []
vocab_config['chainer']['out'] = []

* chainer  
  * chainer.pipe -- 是一个最后运行组件的列表。在这里，您可以指定将哪种顺序和哪种类型的数据提供给组件。  
  
我们的管道由一个组件组成 -- “default_vocab”。

### 组件配置
组件配置是总模型配置的一部分(如上所述)。  
  
任何组件的配置都包含以下必需的参数:  
  
* name -- 组件的名称(它是到python组件实现的链接)  
* save_path -- 保存组件的路径(有时不需要，例如分词器)  
* load_path -- 加载组件的路径(有时不需要，例如分词器)  
  
和以下可选的参数:  
  
* id -- 组件的引用名
* ref --先前初始化的组件的“id”可以代替“name”  
* fit_on -- 要匹配的数据字段列表(调用组件的__fit__()方法)
* in -- 预测期间输入的数据字段列表  
* out -- 预测期间输出的数据字段列表  
  
  
“default_vocab”组件也有其独特的参数:
* level -- 要操作的级别('token'级别和'char'(字符)级别可用)
* tokenizer -- 如果输入是一个字符串，那么它将由分词器来进行分词，是可选参数 


In [24]:
vocab_comp_config = {
    'name': 'default_vocab',
    'save_path': 'vocabs/token.dict',
    'load_path': 'vocabs/token.dict',
    'fit_on': ['utterance'],
    'level': 'token',
    'tokenizer': {'name': 'split_tokenizer'},
    'main': True
}

In [25]:
vocab_config['chainer']['pipe'] = [
    vocab_comp_config
]

In [26]:
vocab_config

{'dataset_reader': {'name': 'dstc2_v2_reader', 'data_path': 'dstc2_v2'},
 'dataset_iterator': {'name': 'dialog_iterator'},
 'metadata': {'download': [{'url': 'http://files.deeppavlov.ai/datasets/dstc2_v2.tar.gz',
    'subdir': 'dstc2_v2'}]},
 'train': {},
 'chainer': {'in': ['utterance'],
  'in_y': [],
  'out': [],
  'pipe': [{'name': 'default_vocab',
    'save_path': 'vocabs/token.dict',
    'load_path': 'vocabs/token.dict',
    'fit_on': ['utterance'],
    'level': 'token',
    'tokenizer': {'name': 'split_tokenizer'},
    'main': True}]}}

In [27]:
json.dump(vocab_config, open("gobot/vocab_config.json", 'wt'))# 生成json文件

下载“dstc2_v2”数据集，使用deeppavlov.deep_download脚本(你只需要做一次):

In [28]:
from deeppavlov.download import deep_download # it is called "deep" in honor of "Deep Pavlov"

deep_download(['--config', 'gobot/vocab_config.json'])

2018-08-22 16:45:16.40 INFO in 'deeppavlov.download'['download'] at line 142: Downloading...
2018-08-22 16:45:16.44 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 205: Starting new HTTP connection (1): files.deeppavlov.ai:80
2018-08-22 16:45:16.343 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 393: http://files.deeppavlov.ai:80 "GET /datasets/dstc2_v2.tar.gz HTTP/1.1" 302 None
2018-08-22 16:45:16.353 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 205: Starting new HTTP connection (1): 202.112.144.244:80
2018-08-22 16:45:16.377 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 393: http://202.112.144.244:80 "GET /files/6195000001D76870/files.deeppavlov.ai/datasets/dstc2_v2.tar.gz HTTP/1.1" 200 506300
2018-08-22 16:45:16.380 INFO in 'deeppavlov.core.data.utils'['utils'] at line 65: Downloading from http://files.deeppavlov.ai/datasets/dstc2_v2.tar.gz to /home/jiahang/DeepPavlov/download/dstc2_v2.tar.gz
100%|██████████| 506k/506k [00:00<00:

所有数据和模型都保存到deeppavlov模块的根目录+../download (DEEPPAVLOV_ROOT/../download/)路径下。

In [29]:
dstc2_v2_path = deeppavlov.__path__[0] + '/../download/dstc2_v2'

数据下载到dstc2_v2_path:

In [30]:
# The command will only work for linux, do not panic otherwise -- it isn't something crucially important.
# You can further just comment bash commands.
!echo "> ls $dstc2_v2_path"
!ls $dstc2_v2_path

> ls /home/jiahang/DeepPavlov/deeppavlov/../download/dstc2_v2
dstc2-templates.txt  dstc2-tst.jsonlist  resto.sqlite
dstc2-trn.jsonlist   dstc2-val.jsonlist


构建我们的词汇表：

In [31]:
from deeppavlov.core.commands.train import train_evaluate_model_from_config

train_evaluate_model_from_config("gobot/vocab_config.json")

2018-08-22 16:45:17.174 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 214: [loading dialogs from /home/jiahang/DeepPavlov/download/dstc2_v2/dstc2-trn.jsonlist]
2018-08-22 16:45:17.518 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 214: [loading dialogs from /home/jiahang/DeepPavlov/download/dstc2_v2/dstc2-val.jsonlist]
2018-08-22 16:45:17.716 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 214: [loading dialogs from /home/jiahang/DeepPavlov/download/dstc2_v2/dstc2-tst.jsonlist]
2018-08-22 16:45:17.970 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 150: [saving vocabulary to /home/jiahang/DeepPavlov/download/vocabs/token.dict]
2018-08-22 16:45:17.973 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 162: [loading vocabulary from /home/jiahang/DeepPavlov/download/vocabs/token.dict]
2018-08-22 16:45:17.974 INFO in 'deeppavlov.core.commands.train'['train'] at line 176: Testing the best saved model


{"valid": {"eval_examples_count": 575, "metrics": {"accuracy": 0.0}, "time_spent": "0:00:01"}}
{"test": {"eval_examples_count": 576, "metrics": {"accuracy": 0.0}, "time_spent": "0:00:01"}}


词汇表是建立在数据基础上并保存到磁盘上的。  
save_path = 'vocabs/token.dict'  
组件文件被保存到 DEEPPAVLOV_ROOT/../download/vocabs/token.dict.

In [32]:
vocabs_path = deeppavlov.__path__[0] + '/../download/vocabs'

In [33]:
!echo "> ls $vocabs_path"
!ls $vocabs_path

> ls /home/jiahang/DeepPavlov/deeppavlov/../download/vocabs
token.dict


这是已经保存好的“token.dict”的内容:

In [34]:
!echo "> head $vocabs_path/token.dict"
!head $vocabs_path/token.dict

> head /home/jiahang/DeepPavlov/deeppavlov/../download/vocabs/token.dict
cheap	391
restaurant	1179
any	259
south	306
address	781
phone	819
number	825
thank	998
you	1055
good	891


### 使用训练好的组件
我们可以通过初始化build_model_from_config来构建词汇表。  
我们需要在组件配置中添加in和out这两个元素(以了解在预测期间输入和输出是什么):  
* in -- 预测期间输入的数据字段列表  
* out -- 预测期间输出的数据字段列表  

In [35]:
vocab_comp_config['in'] = ['utterance']
vocab_comp_config['out'] = ['utterance_token_indices']

vocab_config['chainer']['pipe'] = [
    vocab_comp_config
]
vocab_config['chainer']['out'] = ['utterance_token_indices']

In [36]:
from deeppavlov.core.commands.infer import build_model_from_config

model = build_model_from_config(vocab_config)

2018-08-22 16:45:18.541 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 162: [loading vocabulary from /home/jiahang/DeepPavlov/download/vocabs/token.dict]


模型需要一个样本列表(批次)作为输入。

In [37]:
model(['hi'])

[141]

## gobot_dstc2_simple模型
现在让我们训练一个简单的面向目标的机器人:

In [38]:
from deeppavlov.download import deep_download
from deeppavlov.core.commands.train import train_evaluate_model_from_config
from deeppavlov.core.commands.infer import build_model_from_config

"dataset_reader", "dataset_iterator"和"metadata" 都是和词汇表相同的

In [39]:
simple_config = {}

simple_config['dataset_reader'] = dstc2_reader_comp_config
simple_config['dataset_iterator'] = dialog_iterator_comp_config
simple_config['metadata'] = {}
simple_config['metadata']['download'] = [
    dstc2_download_config
]

In [40]:
simple_config

{'dataset_reader': {'name': 'dstc2_v2_reader', 'data_path': 'dstc2_v2'},
 'dataset_iterator': {'name': 'dialog_iterator'},
 'metadata': {'download': [{'url': 'http://files.deeppavlov.ai/datasets/dstc2_v2.tar.gz',
    'subdir': 'dstc2_v2'}]}}

这里有一个字典' X '包含上下文'text'，'intents'，'db_result'，'prev_resp_act'  
这里有一个字典" Y "包含响应的’act‘和’text‘  
这里所作的预测'y_predicted'只是'text'  

In [41]:
simple_config['chainer'] = {}
simple_config['chainer']['in'] = ['x']
simple_config['chainer']['in_y'] =['y']
simple_config['chainer']['out'] = ['y_predicted']

机器人由(pipe部分)两部分组成:
* default_vocab（DefaultVocabulary）组件  
  * 从用户话语中记下所有token。  
  * DefaultVocabulary.__call__ 方法,输入token批次，输出他们的索引。  
  
Vocabulary组件和之前一样，但是需要加上构建的引用,要用到id  
* id -- 组件的引用名


In [42]:
vocab_comp_config = {
    'name': 'default_vocab',
    'id': 'token_vocab',
    'load_path': 'vocabs/token.dict',
    'save_path': 'vocabs/token.dict',
    'fit_on': ['x'],
    'level': 'token',
    'tokenizer': {'name': 'split_tokenizer'}
}


向chainer添加vocabulary组件

In [43]:
simple_config['chainer']['pipe'] = []
simple_config['chainer']['pipe'].append(vocab_comp_config)

* go_bot (or GoalOrientedBot)组件  
  * 调用slot_filler方法，用于输出用户的话语中所提及的槽  
      （例如，"i want cheap food" -> {'pricerange': 'cheap'}） *
  * 用tracker(DialogStateTracker)更新对话状态  
      （例如，如果旧状态是{'location': 'north'}，现在的槽是{'pricerange': 'cheap'}，那么新的对话状态就是{'location': 'north', 'pricerange': 'cheap'}）
  * 将用户的字符串格式的话语用tokenizer转化成token  
      (例如， "hi, i want some cheap food" -> \['hi', ',', 'i', 'want', 'some', 'cheap', 'food'\])
  * 然后使用bow_embedder(如果不是None的话)和word_vocab将token嵌入词库中  
      (例如，"cheap" -> \[1, 0, 0, 0, .., 0\])
  * 将话语嵌入连续的embedder(如果不是None的话)中，作为嵌入话语token的一种方式  
      (例如，"i" -> \[0.1231, 0.23423, .., 0.03489\])
  * 连接嵌入并将其作为输入传递给循环神经网络(RNN)
  * 训练RNN(以LongShortTermMemory (LSTM)为核心),输出动作标签(action)
  * 使用template_path和template_type加载模板(从标签映射到字符串),将动作标签转换为字符串  
      (例如，"bye_msg" -> "You are welcome!")
  * 用对话状态中的槽值填充结果字符串  
      （例如，如果对话状态是 {'pricerange': 'cheap'}并且输出字符串格式是 "There are no restaurants in a #pricerange pricerange"，那么输出的结果响应就是"There are no restaurants in a cheap pricerange"）

In [44]:
bot_comp_config = {
    'name': 'go_bot',
    'in': ['x'],
    'in_y': ['y'],
    'out': ['y_predicted'],
    'word_vocab': None,
    'bow_embedder': {"name": "bow"},
    'embedder': None,
    'slot_filler': None,
    'template_path': 'dstc2_v2/dstc2-templates.txt',
    'template_type': 'DualTemplate',
    'database': None,
    'api_call_action': 'api_call',
    'network_parameters': {
      'load_path': 'gobot_dstc2_simple/model',
      'save_path': 'gobot_dstc2_simple/model',
      'dense_size': 64,
      'hidden_size': 128,
      'learning_rate': 0.002,
      'attention_mechanism': None
    },
    'tokenizer': {'name': 'stream_spacy_tokenizer',
                  'lowercase': False},
    'tracker': {'name': 'featurized_tracker',
                'slot_names': ['pricerange', 'this', 'area', 'food', 'name']},
    'main': True,
    'debug': False
}

这是我们用引用的方式来使用vocabulary：

In [45]:
bot_comp_config['word_vocab'] = '#token_vocab'

声明槽填充组件。我们假设槽填充器已经经过训练，并通过引用它的配置来使用它。

In [46]:
slot_filler_comp_config = {
    'config_path': deeppavlov.__path__[0] + '/../deeppavlov/configs/ner/slotfill_dstc2.json'
}

向机器人组件中添加槽填充器：

In [47]:
bot_comp_config['slot_filler'] = slot_filler_comp_config

向pipe中添加bot_comp_config：

In [48]:
simple_config['chainer']['pipe'].append(bot_comp_config)

神经网络(在机器人中)是在不同时间段进行训练的，需要批量的数据。  
这就是为什么我们现在用训练参数填充“train”部分。  
* train -- 训练过程配置  
  * train.batch_size：一批样本中样本数量(在一个训练步骤中他们进入神经网络)
  * train.epochs：在训练期间对数据集进行的迭代次数
  * train.log_every_n_batches和train.log_every_n_epochs：控制日志消息的频率
  * train.metrics：用于验证我们的性能的度量列表
  * train.val_every_n_batches 和 train.val_every_n_epochs：描述我们多久计算一次有效数据分割的度量
  * train.validation_patience:我们能够允许的有效数据上没有度量改进的一些迭代次数

In [49]:
simple_bot_train_config = {
    'batch_size': 4,
    'epochs': 2,
    'log_every_n_batches': -1,
    'log_every_n_epochs': 1,
    'metrics': ['per_item_dialog_accuracy'],
    'val_every_n_epochs': 1,
    'validation_patience': 20
}

In [50]:
simple_config['train'] = simple_bot_train_config

In [51]:
json.dump(simple_config, open("gobot/simple_config.json", 'wt'))

train.epochs现在设为“2”，如果你想训练更聪明的模型，以应该增加它（推荐设置10-200个迭代次数）

In [57]:
deep_download(['--config', slot_filler_comp_config['config_path']])

2018-08-22 16:51:17.125 INFO in 'deeppavlov.download'['download'] at line 142: Downloading...
2018-08-22 16:51:17.131 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 205: Starting new HTTP connection (1): lnsigo.mipt.ru:80
2018-08-22 16:51:17.457 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 393: http://lnsigo.mipt.ru:80 "GET /export/deeppavlov_data/slotfill_dstc2.tar.gz HTTP/1.1" 302 None
2018-08-22 16:51:17.467 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 205: Starting new HTTP connection (1): 202.112.144.234:80
2018-08-22 16:51:17.478 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 393: http://202.112.144.234:80 "GET /files/1084000001D81D32/lnsigo.mipt.ru/export/deeppavlov_data/slotfill_dstc2.tar.gz HTTP/1.1" 200 640674
2018-08-22 16:51:17.486 INFO in 'deeppavlov.core.data.utils'['utils'] at line 65: Downloading from http://lnsigo.mipt.ru/export/deeppavlov_data/slotfill_dstc2.tar.gz to /home/jiahang/DeepPavlov/slotfill_dstc2.tar.g

In [59]:
train_evaluate_model_from_config("gobot/simple_config.json")

2018-08-22 16:54:12.488 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 214: [loading dialogs from /home/jiahang/DeepPavlov/download/dstc2_v2/dstc2-trn.jsonlist]
2018-08-22 16:54:12.640 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 214: [loading dialogs from /home/jiahang/DeepPavlov/download/dstc2_v2/dstc2-val.jsonlist]
2018-08-22 16:54:13.9 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 214: [loading dialogs from /home/jiahang/DeepPavlov/download/dstc2_v2/dstc2-tst.jsonlist]
2018-08-22 16:54:13.135 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 162: [loading vocabulary from /home/jiahang/DeepPavlov/download/vocabs/token.dict]
2018-08-22 16:54:13.159 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 150: [saving vocabulary to /home/jiahang/DeepPavlov/download/vocabs/token.dict]
2018-08-22 16:54:13.162 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 94: [loading vocabulary fro

{"train": {"epochs_done": 1, "batches_seen": 242, "train_examples_seen": 967, "metrics": {"per_item_dialog_accuracy": 0.4012}, "time_spent": "0:02:26"}}


2018-08-22 16:57:30.52 INFO in 'deeppavlov.core.commands.train'['train'] at line 388: New best per_item_dialog_accuracy of 0.3967
2018-08-22 16:57:30.53 INFO in 'deeppavlov.core.commands.train'['train'] at line 390: Saving model
2018-08-22 16:57:30.54 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 49: [saving model to /home/jiahang/DeepPavlov/download/gobot_dstc2_simple/model]
2018-08-22 16:57:30.150 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 297: [saving parameters to /home/jiahang/DeepPavlov/download/gobot_dstc2_simple/model.json]


{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.3967}, "time_spent": "0:03:15", "epochs_done": 1, "batches_seen": 242, "train_examples_seen": 967, "impatience": 0, "patience_limit": 20}}


2018-08-22 16:59:50.907 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 315: Updating global step, learning rate = 0.002000.


{"train": {"epochs_done": 2, "batches_seen": 484, "train_examples_seen": 1934, "metrics": {"per_item_dialog_accuracy": 0.5338}, "time_spent": "0:05:36"}}


2018-08-22 17:00:37.778 INFO in 'deeppavlov.core.commands.train'['train'] at line 388: New best per_item_dialog_accuracy of 0.4306
2018-08-22 17:00:37.779 INFO in 'deeppavlov.core.commands.train'['train'] at line 390: Saving model
2018-08-22 17:00:37.779 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 49: [saving model to /home/jiahang/DeepPavlov/download/gobot_dstc2_simple/model]
2018-08-22 17:00:37.854 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 297: [saving parameters to /home/jiahang/DeepPavlov/download/gobot_dstc2_simple/model.json]
2018-08-22 17:00:37.859 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 162: [loading vocabulary from /home/jiahang/DeepPavlov/download/vocabs/token.dict]
2018-08-22 17:00:37.860 WARNING in 'deeppavlov.core.commands.infer'['infer'] at line 43: No "save_path" parameter for the go_bot component, so "load_path" will not be renewed
2018-08-22 17:00:37.863 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at l

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.4306}, "time_spent": "0:06:23", "epochs_done": 2, "batches_seen": 484, "train_examples_seen": 1934, "impatience": 0, "patience_limit": 20}}


2018-08-22 17:00:38.583 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 40: [loading model from /home/jiahang/DeepPavlov/download/slotfill_dstc2/model]
2018-08-22 17:00:38.597 INFO in 'tensorflow'['tf_logging'] at line 116: Restoring parameters from /home/jiahang/DeepPavlov/download/slotfill_dstc2/model
2018-08-22 17:00:39.676 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 56: No load path is set for GoalOrientedBot!
2018-08-22 17:00:39.677 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 67: [loading templates from /home/jiahang/DeepPavlov/download/dstc2_v2/dstc2-templates.txt]
2018-08-22 17:00:39.678 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 70: 46 templates loaded
2018-08-22 17:00:39.679 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 94: Calculated input size for `GoalOrientedBotNetwork` is 511
2018-08-22 17:00:40.437 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 52: [initializing `GoalOrientedBotNetwork

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.4306}, "time_spent": "0:00:48"}}
{"test": {"eval_examples_count": 576, "metrics": {"per_item_dialog_accuracy": 0.4167}, "time_spent": "0:00:48"}}


让我们和生成的机器人进行对话。“退出”消息引发对话结束。

In [60]:
model = build_model_from_config(simple_config)

2018-08-22 17:02:14.586 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 162: [loading vocabulary from /home/jiahang/DeepPavlov/download/vocabs/token.dict]
2018-08-22 17:02:14.589 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 94: [loading vocabulary from /home/jiahang/DeepPavlov/download/slotfill_dstc2/word.dict]
2018-08-22 17:02:14.592 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 94: [loading vocabulary from /home/jiahang/DeepPavlov/download/slotfill_dstc2/tag.dict]
2018-08-22 17:02:15.624 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 40: [loading model from /home/jiahang/DeepPavlov/download/slotfill_dstc2/model]
2018-08-22 17:02:15.637 INFO in 'tensorflow'['tf_logging'] at line 116: Restoring parameters from /home/jiahang/DeepPavlov/download/slotfill_dstc2/model
2018-08-22 17:02:15.981 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 56: No load path is set for GoalOrientedBot!
2018-08-22 17:02:

In [61]:
model(['hi, i want some cheap italian food in the north of town'])

2018-08-22 17:02:26.905 WARNING in 'deeppavlov.models.go_bot.bot'['bot'] at line 343: No database specified.
2018-08-22 17:02:26.906 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 344: Made api_call with {'pricerange': 'cheap', 'food': 'italian', 'area': 'north'}, got 0 results.


['Sorry there is no italian restaurant in the north of town.']

In [62]:
model(['thanks, bye'])

2018-08-22 17:02:29.769 WARNING in 'deeppavlov.models.go_bot.bot'['bot'] at line 343: No database specified.
2018-08-22 17:02:29.770 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 344: Made api_call with {'pricerange': 'cheap', 'food': 'italian', 'area': 'north'}, got 0 results.


['Sorry there is no italian restaurant in the north of town.']

In [63]:
model.reset() # resetting dialog context to start a new one

In [64]:
# if the cell is running, please do not run other cells in parallel -- there is a possibility of a hangup

utterance = ""
while utterance != 'exit':
    print(">> " + model([utterance])[0])
    utterance = input(':: ')

2018-08-22 17:02:33.441 WARNING in 'deeppavlov.models.go_bot.bot'['bot'] at line 343: No database specified.
2018-08-22 17:02:33.442 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 344: Made api_call with {'pricerange': 'cheap', 'food': 'italian', 'area': 'north'}, got 0 results.


>> Sorry there is no italian restaurant in the north of town.
:: i want french food


2018-08-22 17:03:01.118 WARNING in 'deeppavlov.models.go_bot.bot'['bot'] at line 343: No database specified.
2018-08-22 17:03:01.119 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 344: Made api_call with {'pricerange': 'cheap', 'food': 'french', 'area': 'north'}, got 0 results.


>> Sorry there is no french restaurant in the north of town.
:: ok bye


2018-08-22 17:03:13.734 WARNING in 'deeppavlov.models.go_bot.bot'['bot'] at line 343: No database specified.
2018-08-22 17:03:13.735 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 344: Made api_call with {'pricerange': 'cheap', 'food': 'french', 'area': 'north'}, got 0 results.


>> Sorry there is no french restaurant in the north of town.
:: exit


这个模型无法填补一些槽。例如：餐厅的#address, #phone, #postcode无法从话语中推断出来。  
需要一份可用餐厅的列表。

## gobot_dstc2_db模型

现在让我们添加一个包含餐馆的数据库，并训练一个新模型:
初始化新的配置：

In [65]:
db_config = copy.deepcopy(simple_config)

db_config['chainer']['pipe'] = []

创建数据库组件配置:  

In [66]:
db_comp_config = {
    'name': 'sqlite_database',
    'id': 'restaurant_database', 
    'save_path': 'dstc2_v2/resto.sqlite',
    'primary_keys': ['name'],
    'table_name': 'mytable'
}

在管道中添加vocab和数据库组件:

In [67]:
db_config['chainer']['pipe'].append(vocab_comp_config)
db_config['chainer']['pipe'].append(db_comp_config)

初始化机器人组件配置:

In [68]:
bot_with_db_comp_config = copy.deepcopy(bot_comp_config)

警告：训练一个新的模型时，不要忘了改变神经网络中的load_path和save_path这两个变量，否则以前模型的文件将会被覆盖。

In [69]:
bot_with_db_comp_config['network_parameters']['load_path'] = 'gobot_dstc2_db/model'
bot_with_db_comp_config['network_parameters']['save_path'] = 'gobot_dstc2_db/model'

向机器人组件配置添加数据库：

In [70]:
bot_with_db_comp_config['database'] = '#restaurant_database'

向管道添加机器人组件:

In [71]:
db_config['chainer']['pipe'].append(bot_with_db_comp_config)

In [72]:
json.dump(db_config, open("gobot/db_config.json", 'wt'))

新模型在更新对话状态中不仅仅可以更新用户提及到的命名实体部分，还可以更新餐厅数据库获得到的餐厅信息。  
  
  
模型有个特殊的动作叫api_call_action，它可以以当前对话状态向sql数据库发起请求，从而接收单个匹配餐馆的信息。  
  
  
所以现在像#address， #phone和#postcode这样的槽可以在机器人的回复中填写。

In [73]:
train_evaluate_model_from_config("gobot/db_config.json")

2018-08-22 17:03:39.959 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 214: [loading dialogs from /home/jiahang/DeepPavlov/download/dstc2_v2/dstc2-trn.jsonlist]
2018-08-22 17:03:40.367 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 214: [loading dialogs from /home/jiahang/DeepPavlov/download/dstc2_v2/dstc2-val.jsonlist]
2018-08-22 17:03:40.472 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 214: [loading dialogs from /home/jiahang/DeepPavlov/download/dstc2_v2/dstc2-tst.jsonlist]
2018-08-22 17:03:40.672 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 162: [loading vocabulary from /home/jiahang/DeepPavlov/download/vocabs/token.dict]
2018-08-22 17:03:40.693 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 150: [saving vocabulary to /home/jiahang/DeepPavlov/download/vocabs/token.dict]
2018-08-22 17:03:40.697 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 56: No load path is 

{"train": {"epochs_done": 1, "batches_seen": 242, "train_examples_seen": 967, "metrics": {"per_item_dialog_accuracy": 0.4144}, "time_spent": "0:02:24"}}


2018-08-22 17:06:55.381 INFO in 'deeppavlov.core.commands.train'['train'] at line 388: New best per_item_dialog_accuracy of 0.4238
2018-08-22 17:06:55.382 INFO in 'deeppavlov.core.commands.train'['train'] at line 390: Saving model
2018-08-22 17:06:55.382 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 49: [saving model to /home/jiahang/DeepPavlov/download/gobot_dstc2_db/model]
2018-08-22 17:06:55.473 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 297: [saving parameters to /home/jiahang/DeepPavlov/download/gobot_dstc2_db/model.json]


{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.4238}, "time_spent": "0:03:13", "epochs_done": 1, "batches_seen": 242, "train_examples_seen": 967, "impatience": 0, "patience_limit": 20}}


2018-08-22 17:09:18.813 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 315: Updating global step, learning rate = 0.002000.


{"train": {"epochs_done": 2, "batches_seen": 484, "train_examples_seen": 1934, "metrics": {"per_item_dialog_accuracy": 0.5568}, "time_spent": "0:05:36"}}


2018-08-22 17:10:06.849 INFO in 'deeppavlov.core.commands.train'['train'] at line 388: New best per_item_dialog_accuracy of 0.4787
2018-08-22 17:10:06.850 INFO in 'deeppavlov.core.commands.train'['train'] at line 390: Saving model
2018-08-22 17:10:06.851 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 49: [saving model to /home/jiahang/DeepPavlov/download/gobot_dstc2_db/model]
2018-08-22 17:10:06.953 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 297: [saving parameters to /home/jiahang/DeepPavlov/download/gobot_dstc2_db/model.json]
2018-08-22 17:10:06.958 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 162: [loading vocabulary from /home/jiahang/DeepPavlov/download/vocabs/token.dict]
2018-08-22 17:10:06.960 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 53: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2018-08-22 17:10:06.961 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database']

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.4787}, "time_spent": "0:06:24", "epochs_done": 2, "batches_seen": 484, "train_examples_seen": 1934, "impatience": 0, "patience_limit": 20}}


2018-08-22 17:10:07.692 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 40: [loading model from /home/jiahang/DeepPavlov/download/slotfill_dstc2/model]
2018-08-22 17:10:07.706 INFO in 'tensorflow'['tf_logging'] at line 116: Restoring parameters from /home/jiahang/DeepPavlov/download/slotfill_dstc2/model
2018-08-22 17:10:08.662 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 56: No load path is set for GoalOrientedBot!
2018-08-22 17:10:08.663 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 67: [loading templates from /home/jiahang/DeepPavlov/download/dstc2_v2/dstc2-templates.txt]
2018-08-22 17:10:08.664 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 70: 46 templates loaded
2018-08-22 17:10:08.664 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 94: Calculated input size for `GoalOrientedBotNetwork` is 511
2018-08-22 17:10:09.877 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 52: [initializing `GoalOrientedBotNetwork

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.4787}, "time_spent": "0:00:49"}}
{"test": {"eval_examples_count": 576, "metrics": {"per_item_dialog_accuracy": 0.4733}, "time_spent": "0:00:46"}}


In [74]:
model = build_model_from_config(db_config)

2018-08-22 17:11:44.220 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 162: [loading vocabulary from /home/jiahang/DeepPavlov/download/vocabs/token.dict]
2018-08-22 17:11:44.223 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 53: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2018-08-22 17:11:44.224 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 57: Loading database from /home/jiahang/DeepPavlov/download/dstc2_v2/resto.sqlite.
2018-08-22 17:11:44.227 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 94: [loading vocabulary from /home/jiahang/DeepPavlov/download/slotfill_dstc2/word.dict]
2018-08-22 17:11:44.230 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 94: [loading vocabulary from /home/jiahang/DeepPavlov/download/slotfill_dstc2/tag.dict]
2018-08-22 17:11:44.946 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 40: [loading model from /home

In [75]:
# if the cell is running, please do not run other cells in parallel -- there is a possibility of a hangup

model.reset() # starting new dialog

utterance = ""
while utterance != 'exit':
    print(">> " + model([utterance])[0])
    utterance = input(':: ')

>> Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?
:: i want cheap restaurant in the north of town


2018-08-22 17:12:03.50 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 344: Made api_call with {'pricerange': 'cheap', 'area': 'north'}, got 2 results.


>> Da vinci pizzeria is a nice place in the north of town and the prices are cheap.
:: ok, give me their phone number
>> The phone number of da vinci pizzeria is 01223 351707.
:: post code too
>> Da vinci pizzeria is a nice place in the north of town and the prices are cheap.
:: give me their address
>> Sure, da vinci pizzeria is on 20 milton road chesterton.
::  bye
>> Da vinci pizzeria is a nice place in the north of town and the prices are cheap.
:: exit


## gobot_dstc2_emb模型
现在让我们训练一个带有快速文本嵌入的面向目标的机器人:   
  
注意:您需要自己构建一个新的配置  
  
初始化新配置：

In [122]:
emb_config = copy.deepcopy(db_config)

emb_config['chainer']['pipe'] = []

将vocab和数据库组件添加到chainer管道:

In [123]:
emb_config['chainer']['pipe'].append(vocab_comp_config)
emb_config['chainer']['pipe'].append(db_comp_config)

初始化嵌入组件：

In [124]:
embedder_comp_config = {
    'id': 'my_embedder',
    'name': 'fasttext',
    'load_path': 'embeddings/dstc2_fastText_model.bin',
    'save_path': 'embeddings/dstc2_fastText_model.bin',
    'dim': 100
}

In [125]:
# TODO: add embedder component to chainer pipe
emb_config['chainer']['pipe'].append(embedder_comp_config)

初始化机器人组件配置:

In [126]:
bot_with_embedder_comp_config = copy.deepcopy(bot_with_db_comp_config)

bot_with_embedder_comp_config['network_parameters']['load_path'] = 'gobot_dstc2_emb/model'
bot_with_embedder_comp_config['network_parameters']['save_path'] = 'gobot_dstc2_emb/model'

In [127]:
# TODO: add #my_embedder to bot_with_embedder_comp_config
bot_with_embedder_comp_config['embedder'] = '#my_embedder'

In [128]:
# TODO: add bot_with_embedder_comp_config to chainer pipe
emb_config['chainer']['pipe'].append(bot_with_embedder_comp_config)

这些是下载新的需要的数据的url:

In [129]:
embedder_required_data = {
    'url': 'http://files.deeppavlov.ai/deeppavlov_data/embeddings/dstc2_fastText_model.bin',
    'subdir': 'embeddings'
}

In [130]:
# TODO: add embedder download info to emb_config['metadata']
emb_config['metadata'] = {}
emb_config['metadata']['download'] = [
    embedder_required_data
]

In [131]:
json.dump(emb_config, open("gobot/emb_config.json", 'wt'))

目前我们使用的是嵌入式系统，我们需要向metadata.download部分添加一个文件名为dstc2_fastText_model.bin的文件。  
  
  让我们再次运行加载数据。

In [146]:
deep_download(['--config', 'gobot/emb_config.json'])

2018-08-22 18:31:45.570 INFO in 'deeppavlov.download'['download'] at line 142: Downloading...
2018-08-22 18:31:45.574 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 205: Starting new HTTP connection (1): files.deeppavlov.ai:80
2018-08-22 18:31:46.581 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 393: http://files.deeppavlov.ai:80 "GET /deeppavlov_data/embeddings/dstc2_fastText_model.bin HTTP/1.1" 200 800203996
2018-08-22 18:31:46.585 INFO in 'deeppavlov.core.data.utils'['utils'] at line 65: Downloading from http://files.deeppavlov.ai/deeppavlov_data/embeddings/dstc2_fastText_model.bin to /home/jiahang/DeepPavlov/download/embeddings/dstc2_fastText_model.bin

  0%|          | 0.00/800M [00:00<?, ?B/s]
  0%|          | 16.4k/800M [00:00<3:31:25, 63.1kB/s]
  0%|          | 32.8k/800M [00:00<3:36:34, 61.6kB/s]
  0%|          | 49.2k/800M [00:00<3:39:54, 60.6kB/s]
  0%|          | 65.5k/800M [00:01<3:42:23, 60.0kB/s]
  0%|          | 81.9k/800M [00:01<3:44:20, 59.4

  1%|▏         | 11.4M/800M [00:27<34:09, 385kB/s]
  1%|▏         | 11.5M/800M [00:27<36:35, 359kB/s]
  1%|▏         | 11.6M/800M [00:27<31:50, 413kB/s]
  1%|▏         | 11.6M/800M [00:27<36:00, 365kB/s]
  1%|▏         | 11.7M/800M [00:27<30:10, 436kB/s]
  1%|▏         | 11.7M/800M [00:27<32:07, 409kB/s]
  1%|▏         | 11.8M/800M [00:27<28:16, 465kB/s]
  1%|▏         | 11.9M/800M [00:28<29:26, 446kB/s]
  1%|▏         | 12.0M/800M [00:28<25:23, 517kB/s]
  2%|▏         | 12.1M/800M [00:28<27:22, 480kB/s]
  2%|▏         | 12.2M/800M [00:28<23:11, 567kB/s]
  2%|▏         | 12.2M/800M [00:28<24:51, 528kB/s]
  2%|▏         | 12.4M/800M [00:28<21:37, 607kB/s]
  2%|▏         | 12.4M/800M [00:28<22:47, 576kB/s]
  2%|▏         | 12.6M/800M [00:28<20:05, 654kB/s]
  2%|▏         | 12.6M/800M [00:29<21:19, 616kB/s]
  2%|▏         | 12.7M/800M [00:29<18:26, 712kB/s]
  2%|▏         | 12.8M/800M [00:29<20:53, 628kB/s]
  2%|▏         | 13.0M/800M [00:29<17:22, 755kB/s]
  2%|▏         | 13.1M/800M [00

  4%|▎         | 28.5M/800M [00:49<16:35, 775kB/s]
  4%|▎         | 28.6M/800M [00:50<16:06, 798kB/s]
  4%|▎         | 28.7M/800M [00:50<16:09, 796kB/s]
  4%|▎         | 28.8M/800M [00:50<14:21, 895kB/s]
  4%|▎         | 28.9M/800M [00:50<16:16, 790kB/s]
  4%|▎         | 29.1M/800M [00:50<14:31, 884kB/s]
  4%|▎         | 29.2M/800M [00:50<16:30, 779kB/s]
  4%|▎         | 29.3M/800M [00:50<14:40, 875kB/s]
  4%|▎         | 29.4M/800M [00:51<26:29, 485kB/s]
  4%|▎         | 29.8M/800M [00:51<19:45, 650kB/s]
  4%|▎         | 29.9M/800M [00:51<20:05, 639kB/s]
  4%|▍         | 30.0M/800M [00:51<21:21, 601kB/s]
  4%|▍         | 30.1M/800M [00:52<20:11, 635kB/s]
  4%|▍         | 30.2M/800M [00:52<19:23, 662kB/s]
  4%|▍         | 30.3M/800M [00:52<19:14, 667kB/s]
  4%|▍         | 30.4M/800M [00:52<19:36, 654kB/s]
  4%|▍         | 30.5M/800M [00:52<17:39, 726kB/s]
  4%|▍         | 30.6M/800M [00:52<19:20, 663kB/s]
  4%|▍         | 30.7M/800M [00:52<17:29, 733kB/s]
  4%|▍         | 30.8M/800M [00

  6%|▌         | 45.3M/800M [01:13<16:44, 751kB/s]
  6%|▌         | 45.4M/800M [01:13<15:16, 824kB/s]
  6%|▌         | 45.5M/800M [01:13<16:41, 754kB/s]
  6%|▌         | 45.6M/800M [01:13<15:11, 828kB/s]
  6%|▌         | 45.7M/800M [01:14<16:37, 756kB/s]
  6%|▌         | 45.9M/800M [01:14<15:55, 789kB/s]
  6%|▌         | 46.0M/800M [01:14<15:35, 806kB/s]
  6%|▌         | 46.1M/800M [01:14<13:50, 908kB/s]
  6%|▌         | 46.2M/800M [01:14<14:28, 868kB/s]
  6%|▌         | 46.3M/800M [01:14<14:34, 862kB/s]
  6%|▌         | 46.4M/800M [01:14<15:26, 813kB/s]
  6%|▌         | 46.5M/800M [01:14<14:26, 870kB/s]
  6%|▌         | 46.6M/800M [01:15<14:30, 866kB/s]
  6%|▌         | 46.7M/800M [01:15<14:33, 863kB/s]
  6%|▌         | 46.9M/800M [01:15<14:24, 872kB/s]
  6%|▌         | 47.0M/800M [01:15<14:24, 871kB/s]
  6%|▌         | 47.1M/800M [01:15<12:58, 968kB/s]
  6%|▌         | 47.2M/800M [01:15<14:36, 859kB/s]
  6%|▌         | 47.4M/800M [01:15<14:25, 869kB/s]
  6%|▌         | 47.5M/800M [01

  8%|▊         | 64.9M/800M [01:35<13:01, 941kB/s]
  8%|▊         | 65.0M/800M [01:35<11:38, 1.05MB/s]
  8%|▊         | 65.2M/800M [01:35<12:33, 975kB/s] 
  8%|▊         | 65.3M/800M [01:36<11:21, 1.08MB/s]
  8%|▊         | 65.5M/800M [01:36<12:12, 1.00MB/s]
  8%|▊         | 65.6M/800M [01:36<11:20, 1.08MB/s]
  8%|▊         | 65.7M/800M [01:36<20:59, 583kB/s] 
  8%|▊         | 66.0M/800M [01:36<16:45, 730kB/s]
  8%|▊         | 66.2M/800M [01:37<13:48, 886kB/s]
  8%|▊         | 66.4M/800M [01:37<13:53, 880kB/s]
  8%|▊         | 66.5M/800M [01:37<15:27, 791kB/s]
  8%|▊         | 66.7M/800M [01:37<13:04, 935kB/s]
  8%|▊         | 66.8M/800M [01:37<15:52, 770kB/s]
  8%|▊         | 66.9M/800M [01:37<15:16, 800kB/s]
  8%|▊         | 67.0M/800M [01:37<15:36, 783kB/s]
  8%|▊         | 67.2M/800M [01:38<14:34, 838kB/s]
  8%|▊         | 67.3M/800M [01:38<15:08, 807kB/s]
  8%|▊         | 67.4M/800M [01:38<14:13, 859kB/s]
  8%|▊         | 67.5M/800M [01:38<14:46, 827kB/s]
  8%|▊         | 67.6M/80

...

 89%|████████▉ | 714M/800M [15:40<01:34, 912kB/s]
 89%|████████▉ | 714M/800M [15:40<01:39, 863kB/s]
 89%|████████▉ | 715M/800M [15:41<01:33, 920kB/s]
 89%|████████▉ | 715M/800M [15:41<01:38, 867kB/s]
 89%|████████▉ | 715M/800M [15:41<01:25, 999kB/s]
 89%|████████▉ | 715M/800M [15:41<01:40, 852kB/s]
 89%|████████▉ | 715M/800M [15:41<01:32, 920kB/s]
 89%|████████▉ | 715M/800M [15:41<01:35, 886kB/s]
 89%|████████▉ | 715M/800M [15:41<01:30, 936kB/s]
 89%|████████▉ | 716M/800M [15:42<01:32, 914kB/s]
 89%|████████▉ | 716M/800M [15:42<01:27, 962kB/s]
 89%|████████▉ | 716M/800M [15:42<01:30, 935kB/s]
 89%|████████▉ | 716M/800M [15:42<01:26, 979kB/s]
 89%|████████▉ | 716M/800M [15:42<01:28, 946kB/s]
 90%|████████▉ | 716M/800M [15:42<01:22, 1.01MB/s]
 90%|████████▉ | 716M/800M [15:42<01:26, 964kB/s] 
 90%|████████▉ | 717M/800M [15:43<01:21, 1.03MB/s]
 90%|████████▉ | 717M/800M [15:43<01:25, 977kB/s] 
 90%|████████▉ | 717M/800M [15:43<01:18, 1.06MB/s]
 90%|████████▉ | 717M/800M [15:43<01:24, 989k

 92%|█████████▏| 734M/800M [16:01<01:18, 840kB/s]
 92%|█████████▏| 734M/800M [16:01<01:15, 875kB/s]
 92%|█████████▏| 734M/800M [16:01<01:10, 931kB/s]
 92%|█████████▏| 734M/800M [16:02<01:17, 846kB/s]
 92%|█████████▏| 734M/800M [16:02<01:14, 887kB/s]
 92%|█████████▏| 734M/800M [16:02<01:21, 805kB/s]
 92%|█████████▏| 735M/800M [16:02<01:14, 879kB/s]
 92%|█████████▏| 735M/800M [16:02<01:12, 902kB/s]
 92%|█████████▏| 735M/800M [16:02<01:11, 917kB/s]
 92%|█████████▏| 735M/800M [16:02<01:08, 960kB/s]
 92%|█████████▏| 735M/800M [16:02<01:15, 859kB/s]
 92%|█████████▏| 735M/800M [16:03<01:11, 914kB/s]
 92%|█████████▏| 735M/800M [16:03<01:18, 833kB/s]
 92%|█████████▏| 735M/800M [16:03<01:10, 921kB/s]
 92%|█████████▏| 735M/800M [16:03<01:13, 887kB/s]
 92%|█████████▏| 736M/800M [16:03<01:11, 904kB/s]
 92%|█████████▏| 736M/800M [16:03<01:09, 925kB/s]
 92%|█████████▏| 736M/800M [16:03<01:08, 937kB/s]
 92%|█████████▏| 736M/800M [16:03<01:07, 950kB/s]
 92%|█████████▏| 736M/800M [16:03<01:13, 879kB/s]


 94%|█████████▍| 755M/800M [16:23<00:59, 761kB/s]
 94%|█████████▍| 755M/800M [16:23<00:53, 856kB/s]
 94%|█████████▍| 755M/800M [16:24<00:54, 829kB/s]
 94%|█████████▍| 755M/800M [16:24<00:50, 901kB/s]
 94%|█████████▍| 755M/800M [16:24<00:52, 854kB/s]
 94%|█████████▍| 755M/800M [16:24<00:51, 878kB/s]
 94%|█████████▍| 755M/800M [16:24<00:50, 885kB/s]
 94%|█████████▍| 756M/800M [16:24<00:47, 932kB/s]
 94%|█████████▍| 756M/800M [16:24<00:48, 919kB/s]
 94%|█████████▍| 756M/800M [16:25<00:42, 1.04MB/s]
 94%|█████████▍| 756M/800M [16:25<00:49, 895kB/s] 
 94%|█████████▍| 756M/800M [16:25<00:42, 1.04MB/s]
 95%|█████████▍| 756M/800M [16:25<00:47, 928kB/s] 
 95%|█████████▍| 756M/800M [16:25<00:43, 994kB/s]
 95%|█████████▍| 757M/800M [16:25<00:45, 957kB/s]
 95%|█████████▍| 757M/800M [16:26<01:02, 695kB/s]
 95%|█████████▍| 757M/800M [16:26<00:59, 724kB/s]
 95%|█████████▍| 757M/800M [16:26<00:51, 833kB/s]
 95%|█████████▍| 757M/800M [16:26<00:43, 985kB/s]
 95%|█████████▍| 757M/800M [16:26<00:46, 914kB

 97%|█████████▋| 779M/800M [16:50<00:23, 905kB/s]
 97%|█████████▋| 779M/800M [16:50<00:26, 812kB/s]
 97%|█████████▋| 779M/800M [16:50<00:23, 910kB/s]
 97%|█████████▋| 779M/800M [16:50<00:25, 836kB/s]
 97%|█████████▋| 779M/800M [16:50<00:22, 933kB/s]
 97%|█████████▋| 779M/800M [16:50<00:25, 826kB/s]
 97%|█████████▋| 780M/800M [16:51<00:21, 949kB/s]
 97%|█████████▋| 780M/800M [16:51<00:24, 850kB/s]
 97%|█████████▋| 780M/800M [16:51<00:20, 973kB/s]
 97%|█████████▋| 780M/800M [16:51<00:23, 861kB/s]
 97%|█████████▋| 780M/800M [16:51<00:22, 886kB/s]
 98%|█████████▊| 780M/800M [16:51<00:21, 912kB/s]
 98%|█████████▊| 780M/800M [16:52<00:25, 790kB/s]
 98%|█████████▊| 781M/800M [16:52<00:18, 1.02MB/s]
 98%|█████████▊| 781M/800M [16:52<00:19, 992kB/s] 
 98%|█████████▊| 781M/800M [16:52<00:17, 1.08MB/s]
 98%|█████████▊| 781M/800M [16:52<00:18, 1.02MB/s]
 98%|█████████▊| 781M/800M [16:52<00:17, 1.09MB/s]
 98%|█████████▊| 782M/800M [16:53<00:29, 626kB/s] 
 98%|█████████▊| 782M/800M [16:53<00:25, 721

100%|██████████| 800M/800M [17:13<00:00, 774kB/s]2018-08-22 18:48:59.904 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 205: Starting new HTTP connection (1): lnsigo.mipt.ru:80
2018-08-22 18:49:00.200 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 393: http://lnsigo.mipt.ru:80 "GET /export/deeppavlov_data/slotfill_dstc2.tar.gz HTTP/1.1" 302 None
2018-08-22 18:49:00.211 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 205: Starting new HTTP connection (1): 202.112.144.234:80
2018-08-22 18:49:00.241 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 393: http://202.112.144.234:80 "GET /files/1084000001D81D32/lnsigo.mipt.ru/export/deeppavlov_data/slotfill_dstc2.tar.gz HTTP/1.1" 200 640674
2018-08-22 18:49:00.246 INFO in 'deeppavlov.core.data.utils'['utils'] at line 65: Downloading from http://lnsigo.mipt.ru/export/deeppavlov_data/slotfill_dstc2.tar.gz to /home/jiahang/DeepPavlov/slotfill_dstc2.tar.gz

  0%|          | 0.00/641k [00:00<?, ?B/s]

In [147]:
train_evaluate_model_from_config("gobot/emb_config.json")

2018-08-22 18:49:00.764 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 214: [loading dialogs from /home/jiahang/DeepPavlov/download/dstc2_v2/dstc2-trn.jsonlist]
2018-08-22 18:49:01.294 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 214: [loading dialogs from /home/jiahang/DeepPavlov/download/dstc2_v2/dstc2-val.jsonlist]
2018-08-22 18:49:01.397 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 214: [loading dialogs from /home/jiahang/DeepPavlov/download/dstc2_v2/dstc2-tst.jsonlist]
2018-08-22 18:49:01.514 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 162: [loading vocabulary from /home/jiahang/DeepPavlov/download/vocabs/token.dict]
2018-08-22 18:49:01.536 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 150: [saving vocabulary to /home/jiahang/DeepPavlov/download/vocabs/token.dict]
2018-08-22 18:49:01.538 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 56: No load path is 

{"train": {"epochs_done": 1, "batches_seen": 242, "train_examples_seen": 967, "metrics": {"per_item_dialog_accuracy": 0.5843}, "time_spent": "0:02:21"}}


2018-08-22 18:52:12.18 INFO in 'deeppavlov.core.commands.train'['train'] at line 388: New best per_item_dialog_accuracy of 0.4476
2018-08-22 18:52:12.19 INFO in 'deeppavlov.core.commands.train'['train'] at line 390: Saving model
2018-08-22 18:52:12.21 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 49: [saving model to /home/jiahang/DeepPavlov/download/gobot_dstc2_emb/model]
2018-08-22 18:52:12.125 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 297: [saving parameters to /home/jiahang/DeepPavlov/download/gobot_dstc2_emb/model.json]


{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.4476}, "time_spent": "0:03:08", "epochs_done": 1, "batches_seen": 242, "train_examples_seen": 967, "impatience": 0, "patience_limit": 20}}


2018-08-22 18:54:32.146 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 315: Updating global step, learning rate = 0.002000.


{"train": {"epochs_done": 2, "batches_seen": 484, "train_examples_seen": 1934, "metrics": {"per_item_dialog_accuracy": 0.5919}, "time_spent": "0:05:28"}}


2018-08-22 18:55:23.62 INFO in 'deeppavlov.core.commands.train'['train'] at line 388: New best per_item_dialog_accuracy of 0.4649
2018-08-22 18:55:23.63 INFO in 'deeppavlov.core.commands.train'['train'] at line 390: Saving model
2018-08-22 18:55:23.63 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 49: [saving model to /home/jiahang/DeepPavlov/download/gobot_dstc2_emb/model]
2018-08-22 18:55:23.137 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 297: [saving parameters to /home/jiahang/DeepPavlov/download/gobot_dstc2_emb/model.json]
2018-08-22 18:55:23.143 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 162: [loading vocabulary from /home/jiahang/DeepPavlov/download/vocabs/token.dict]
2018-08-22 18:55:23.145 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 53: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2018-08-22 18:55:23.146 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] 

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.4649}, "time_spent": "0:06:19", "epochs_done": 2, "batches_seen": 484, "train_examples_seen": 1934, "impatience": 0, "patience_limit": 20}}


2018-08-22 18:55:33.174 WARNING in 'deeppavlov.core.commands.infer'['infer'] at line 43: No "save_path" parameter for the go_bot component, so "load_path" will not be renewed
2018-08-22 18:55:33.177 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 94: [loading vocabulary from /home/jiahang/DeepPavlov/download/slotfill_dstc2/word.dict]
2018-08-22 18:55:33.181 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 94: [loading vocabulary from /home/jiahang/DeepPavlov/download/slotfill_dstc2/tag.dict]
2018-08-22 18:55:33.942 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 40: [loading model from /home/jiahang/DeepPavlov/download/slotfill_dstc2/model]
2018-08-22 18:55:33.958 INFO in 'tensorflow'['tf_logging'] at line 116: Restoring parameters from /home/jiahang/DeepPavlov/download/slotfill_dstc2/model
2018-08-22 18:55:34.403 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 56: No load path is set for GoalOrientedBot!
201

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.4653}, "time_spent": "0:00:52"}}
{"test": {"eval_examples_count": 576, "metrics": {"per_item_dialog_accuracy": 0.4533}, "time_spent": "0:00:52"}}


In [148]:
model = build_model_from_config(emb_config)

2018-08-22 18:57:17.812 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 162: [loading vocabulary from /home/jiahang/DeepPavlov/download/vocabs/token.dict]
2018-08-22 18:57:17.817 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 53: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2018-08-22 18:57:17.818 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 57: Loading database from /home/jiahang/DeepPavlov/download/dstc2_v2/resto.sqlite.
2018-08-22 18:57:17.819 INFO in 'deeppavlov.models.embedders.fasttext_embedder'['fasttext_embedder'] at line 69: [loading embeddings from `/home/jiahang/DeepPavlov/download/embeddings/dstc2_fastText_model.bin`]
2018-08-22 18:57:18.814 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 94: [loading vocabulary from /home/jiahang/DeepPavlov/download/slotfill_dstc2/word.dict]
2018-08-22 18:57:18.817 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] 

In [149]:
# if the cell is running, please do not run other cells in parallel -- there is a possibility of a hangup

model.reset() # starting new dialog

utterance = ""
while utterance != 'exit':
    print(">> " + model([utterance])[0])
    utterance = input(':: ')

>> Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?
::  i want cheap restaurant in the north of town


2018-08-22 21:49:49.490 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 344: Made api_call with {'pricerange': 'cheap', 'area': 'north'}, got 2 results.


>> Da vinci pizzeria is a nice place in the north of town and the prices are cheap.
:: ok, give me their phone number
>> Da vinci pizzeria is a nice place in the north of town and the prices are cheap.
:: give me their address
>> Da vinci pizzeria is a nice place in the north of town and the prices are cheap.
:: give me their phone number
>> The phone number of da vinci pizzeria is 01223 351707.
:: exit


## gobot_dstc2_full模型
现在让我们来训练一个非常聪明的面向目标的机器人，它在输入嵌入系统基础上上使用一种注意机制。  
（更多请见： https://medium.com/syncedreview/a-brief-overview-of-attention-mechanism-13c578ba9129 ）  
  
  初始化新的配置：

In [157]:
emb_config = json.load(open("gobot/emb_config.json", 'rt'))

full_config = copy.deepcopy(emb_config)
full_config['chainer']['pipe'] = [
    vocab_comp_config,
    db_comp_config,
    embedder_comp_config
]


初始化机器人组件配置：

In [158]:
bot_with_emb_comp_config = emb_config['chainer']['pipe'][-1]
bot_with_attn_comp_config = copy.deepcopy(bot_with_emb_comp_config)

bot_with_attn_comp_config['network_parameters']['load_path'] = 'gobot_dstc2_full/model'
bot_with_attn_comp_config['network_parameters']['save_path'] = 'gobot_dstc2_full/model'

向机器人的增加注意力机制:

In [159]:
attention_mechanism_config = {
    'action_as_key': True,
    'depth': 3,
    'hidden_size': 32,
    'max_num_tokens': 100,
    'projected_align': False,
    'type': 'cs_bahdanau'
}

In [160]:
bot_with_attn_comp_config['network_parameters']['attention_mechanism'] = attention_mechanism_config

将机器人组件加入到pipe中：

In [161]:
full_config['chainer']['pipe'].append(bot_with_attn_comp_config)

In [162]:
json.dump(full_config, open("gobot/full_config.json", 'wt'))

In [164]:
train_evaluate_model_from_config ("gobot/full_config.json")

2018-08-22 22:00:50.629 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 214: [loading dialogs from /home/jiahang/DeepPavlov/download/dstc2_v2/dstc2-trn.jsonlist]
2018-08-22 22:00:50.938 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 214: [loading dialogs from /home/jiahang/DeepPavlov/download/dstc2_v2/dstc2-val.jsonlist]
2018-08-22 22:00:51.71 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 214: [loading dialogs from /home/jiahang/DeepPavlov/download/dstc2_v2/dstc2-tst.jsonlist]
2018-08-22 22:00:51.264 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 162: [loading vocabulary from /home/jiahang/DeepPavlov/download/vocabs/token.dict]
2018-08-22 22:00:51.295 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 150: [saving vocabulary to /home/jiahang/DeepPavlov/download/vocabs/token.dict]
2018-08-22 22:00:51.297 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 56: No load path is s

{"train": {"epochs_done": 1, "batches_seen": 242, "train_examples_seen": 967, "metrics": {"per_item_dialog_accuracy": 0.6151}, "time_spent": "0:03:45"}}


2018-08-22 22:06:14.253 INFO in 'deeppavlov.core.commands.train'['train'] at line 388: New best per_item_dialog_accuracy of 0.454
2018-08-22 22:06:14.254 INFO in 'deeppavlov.core.commands.train'['train'] at line 390: Saving model
2018-08-22 22:06:14.255 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 49: [saving model to /home/jiahang/DeepPavlov/download/gobot_dstc2_full/model]
2018-08-22 22:06:14.999 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 297: [saving parameters to /home/jiahang/DeepPavlov/download/gobot_dstc2_full/model.json]


{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.454}, "time_spent": "0:05:03", "epochs_done": 1, "batches_seen": 242, "train_examples_seen": 967, "impatience": 0, "patience_limit": 20}}


2018-08-22 22:09:51.597 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 315: Updating global step, learning rate = 0.002000.


{"train": {"epochs_done": 2, "batches_seen": 484, "train_examples_seen": 1934, "metrics": {"per_item_dialog_accuracy": 0.6301}, "time_spent": "0:08:41"}}


2018-08-22 22:11:10.684 INFO in 'deeppavlov.core.commands.train'['train'] at line 388: New best per_item_dialog_accuracy of 0.4653
2018-08-22 22:11:10.685 INFO in 'deeppavlov.core.commands.train'['train'] at line 390: Saving model
2018-08-22 22:11:10.686 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 49: [saving model to /home/jiahang/DeepPavlov/download/gobot_dstc2_full/model]
2018-08-22 22:11:11.0 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 297: [saving parameters to /home/jiahang/DeepPavlov/download/gobot_dstc2_full/model.json]
2018-08-22 22:11:11.3 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 162: [loading vocabulary from /home/jiahang/DeepPavlov/download/vocabs/token.dict]
2018-08-22 22:11:11.20 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 53: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2018-08-22 22:11:11.22 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] a

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.4653}, "time_spent": "0:10:00", "epochs_done": 2, "batches_seen": 484, "train_examples_seen": 1934, "impatience": 0, "patience_limit": 20}}


2018-08-22 22:11:18.969 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 94: [loading vocabulary from /home/jiahang/DeepPavlov/download/slotfill_dstc2/word.dict]
2018-08-22 22:11:18.996 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 94: [loading vocabulary from /home/jiahang/DeepPavlov/download/slotfill_dstc2/tag.dict]
2018-08-22 22:11:19.769 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 40: [loading model from /home/jiahang/DeepPavlov/download/slotfill_dstc2/model]
2018-08-22 22:11:19.783 INFO in 'tensorflow'['tf_logging'] at line 116: Restoring parameters from /home/jiahang/DeepPavlov/download/slotfill_dstc2/model
2018-08-22 22:11:20.542 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 56: No load path is set for GoalOrientedBot!
2018-08-22 22:11:20.543 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 67: [loading templates from /home/jiahang/DeepPavlov/download/dstc2_v2/dstc2-templates.txt]
2018-08-

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.4653}, "time_spent": "0:01:18"}}
{"test": {"eval_examples_count": 576, "metrics": {"per_item_dialog_accuracy": 0.4623}, "time_spent": "0:01:18"}}


In [165]:
model = build_model_from_config(full_config)

2018-08-22 22:13:58.693 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 162: [loading vocabulary from /home/jiahang/DeepPavlov/download/vocabs/token.dict]
2018-08-22 22:13:58.747 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 53: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2018-08-22 22:13:58.749 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 57: Loading database from /home/jiahang/DeepPavlov/download/dstc2_v2/resto.sqlite.
2018-08-22 22:13:58.751 INFO in 'deeppavlov.models.embedders.fasttext_embedder'['fasttext_embedder'] at line 69: [loading embeddings from `/home/jiahang/DeepPavlov/download/embeddings/dstc2_fastText_model.bin`]
2018-08-22 22:14:00.21 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 94: [loading vocabulary from /home/jiahang/DeepPavlov/download/slotfill_dstc2/word.dict]
2018-08-22 22:14:00.25 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at

In [166]:
# if the cell is running, please do not run other cells in parallel -- there is a possibility of a hangup

model.reset() # starting new dialog

utterance = ""
while utterance != 'exit':
    print(">> " + model([utterance])[0])
    utterance = input(':: ')

>> Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?
:: i want to book cheapest
>> Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?
:: i want cheap restaurant in the north of town


2018-08-22 22:22:29.438 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 344: Made api_call with {'pricerange': 'cheap', 'area': 'north'}, got 2 results.


>> Da vinci pizzeria is a nice place in the north of town and the prices are cheap.
:: tell me its phone number
>> The phone number of da vinci pizzeria is 01223 351707.
:: tell me its address
>> Sure, da vinci pizzeria is on 20 milton road chesterton.
:: bye
>> Da vinci pizzeria is a nice place in the north of town and the prices are cheap.
:: exit


## 训练和推断一个组件的另一种方式

让我们构建一个响应token的词汇表，但不使用deeppavlov的脚本（不用train_evaluate_model_from_config 和 build_model_from_config）


In [168]:
from deeppavlov.core.data.vocab import DefaultVocabulary
from deeppavlov.dataset_readers.dstc2_reader import DSTC2Version2DatasetReader
from deeppavlov.dataset_iterators.dialog_iterator import DialogDatasetIterator

初始化一个DefaultVocabulary类:

In [169]:
y_vocab = DefaultVocabulary(level='token', 
                            load_path='vocabs/y_token.dict', # path is relative to DEEPPAVLOV_ROOT/../download/ 
                            save_path='vocabs/y_token.dict',
                            tokenizer=lambda s_batch: [s.split() for s in s_batch])

任何训练好的的部件的重要方法是:  
* __init__(self, args, *kwargs)：  
  初始化一个类的实例
* fit(self, data, *args) or train_on_batch(self, batch, *args)：
  适用于完整的数据或对一批数据进行一个训练步骤
* __call__(self, batch, **kwargs)：
  对每批样本进行预测(或推断)

获得数据批次：

In [170]:
data = DSTC2Version2DatasetReader().read(data_path="tmp/my_download_of_dstc2")
data_samples = DialogDatasetIterator(data, seed=1443, shuffle=True).get_instances(data_type='all')
x_list, y_list = data_samples

2018-08-22 22:23:03.360 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 214: [loading dialogs from tmp/my_download_of_dstc2/dstc2-trn.jsonlist]
2018-08-22 22:23:03.543 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 214: [loading dialogs from tmp/my_download_of_dstc2/dstc2-val.jsonlist]
2018-08-22 22:23:03.701 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 214: [loading dialogs from tmp/my_download_of_dstc2/dstc2-tst.jsonlist]


使用y批次(y_list,包含bot响应)构建词汇表:

In [171]:
y_vocab.fit(y_list)

从构建的词汇表推断:

In [172]:
y_vocab(['is', 'the', 'of', 'restaurant', 'hi'])

[43, 3, 26, 5, 0]

 调用y_vocab(batch)和调用y_vocab.__call__(batch)是一样的

In [173]:
y_vocab(['hi']) == y_vocab.__call__(['hi']) == [141]

False